In [1]:
import os
import random
import subprocess
import numpy as np
import pandas as pd
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
from typing import Optional, Callable
import matplotlib.pyplot as plt
from skimage import io, transform
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms.functional as F
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import draw_segmentation_masks
from zipfile import ZipFile
import glob
import torchvision.models as models
import torch.nn as nn

In [2]:
_path_chest = "/home/data/02_SSD4TB/suzy/datasets/public/chest-xray"
_path_dsb = "/home/data/02_SSD4TB/suzy/datasets/public/data-science-bowl-2018"
_path_histo = "/home/data/02_SSD4TB/suzy/datasets/public/histopathologic-cancer-detection"
_path_ranzcr = "/home/data/02_SSD4TB/suzy/datasets/public/ranzcr-clip-catheter-line-classification"
_path_retina = "/home/data/02_SSD4TB/suzy/datasets/public/aptos2019-blindness-detection"

In [2]:
from farabio.models.classification.conv import *

In [5]:
resnet18()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc): Linear(in_features=512, out_features=1000, bias=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): Ba

In [67]:
#18 34 50 101 152
import numpy as np
import torch
import torch.nn as nn
from typing import Union, List, Any, Type, Optional
from farabio.utils.helpers import get_num_parameters

__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152']

cfgs = {
    'resnet18': [2, 2, 2, 2],
    'resnet34': [3, 4, 6, 3],
    'resnet50': [3, 4, 6, 3],
    'resnet101': [3, 4, 23, 3],
    'resnet152': [3, 8, 36, 3]
}


def conv1x1(
    in_planes: int,
    out_planes: int,
    stride: int = 1,
) -> nn.Conv2d:
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=1,
        stride=stride,
        bias=False
    )

def conv3x3(
    in_planes: int,
    out_planes: int,
    stride: int = 1,
    groups: int = 1,
    dilation: int = 1
) -> nn.Conv2d:
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation
    )

            
# Figure 5 of paper
class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        in_planes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None
    ) -> None:
        super(BasicBlock, self).__init__()
              
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.stride = stride
        self.downsample = downsample
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        
        return out

        
class Bottleneck(nn.Module):
    expansion: int = 4
    def __init__(
        self,
        in_planes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None
    ) -> None:
        super(Bottleneck, self).__init__()
        self.conv1 = conv1x1(in_planes, planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes, stride)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = conv1x1(planes, planes * self.expansion)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.stride = stride
        self.downsample = downsample
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        
        out = self.conv3(out)
        out = self.bn3(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)
        
        out += identity
        out = self.relu(out)
        
        return out


class ResNet(nn.Module):
    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        n_classes: int = 1000
    ) -> None:
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * block.expansion, n_classes)
        
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        
        
    def _make_layer(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        planes,
        num_blocks,
        stride: int = 1
    ) -> nn.Sequential:
        downsample = None
        if stride != 1 or self.in_planes != planes*block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.in_planes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion)
        )
        
        layers = []
        layers.append(block(self.in_planes,planes, stride, downsample))
        
        self.in_planes = planes * block.expansion
        for _ in range(1, num_blocks):
            layers.append(block(self.in_planes, planes))

        return nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)

        x = self.fc(x)
        
        return x
    
def _resnet(
    block: Type[Union[BasicBlock, Bottleneck]],
    layers: List[int],
    **kwargs: Any
) -> ResNet:
    model = ResNet(block, layers, **kwargs)
    return model

def resnet18(**kwargs: Any) -> ResNet:
    return _resnet(BasicBlock, cfgs['resnet18'], **kwargs)

def resnet34(**kwargs: Any) -> ResNet:
    return _resnet(BasicBlock, cfgs['resnet34'], **kwargs)

def resnet50(**kwargs: Any) -> ResNet:
    return _resnet(Bottleneck, cfgs['resnet50'], **kwargs)

def resnet101(**kwargs: Any) -> ResNet:
    return _resnet(Bottleneck, cfgs['resnet101'], **kwargs)

def resnet152(**kwargs: Any) -> ResNet:
    return _resnet(Bottleneck, cfgs['resnet152'], **kwargs)

def test(convnet="resnet18"):
    x = torch.randn(1,3,224,224)
    
    tests = { 
        "resnet18": resnet18(),
        "resnet34": resnet34(),
        "resnet50": resnet50(),
        "resnet101": resnet101(),
        "resnet152": resnet152()
    }

    model = tests[convnet]
    y = model(x)
    
    print("Trainable parameters: ", get_num_parameters(model, True))
    print("in shape: ", x.shape, ", out shape: ", y.shape)
    
test("resnet152")

Trainable parameters:  60192808
in shape:  torch.Size([1, 3, 224, 224]) , out shape:  torch.Size([1, 1000])


In [62]:
test("resnet18")

Trainable parameters:  11689512
in shape:  torch.Size([1, 3, 224, 224]) , out shape:  torch.Size([1, 1000])


In [ ]:
11178051

In [ ]:
11689512

In [ ]:
11689512

In [ ]:
11689512

In [ ]:
 60192808

In [19]:
print("hi]")

hi]


In [11]:
def cal_odd():
    print("hery")

print ("First function name: ", type(str(cal_odd.__name__)))

First function name:  <class 'str'>


In [12]:
a = "as"
print(type(a))

<class 'str'>


In [3]:
model = alexnet(n_classes=5)
print(model)

AlexNet(
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (conv): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): Sequential(
      (0): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
    )
    (3): Sequential(
      (0): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
    )
    (4): Sequential(
      (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1,

In [5]:
"""AlexNet

Adapted from https://github.com/pytorch/vision/blob/master/torchvision/models/alexnet.py

Copyright 2021 | farabio
"""
import torch
import torch.nn as nn
from farabio.utils.helpers import get_num_parameters

__all__ = ['AlexNet', 'alexnet']


class AlexNet(nn.Module):
    def __init__(self, n_classes: int = 1000) -> None:
        super(AlexNet, self).__init__()
        
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.dropout = nn.Dropout()
        
        c1 = self._conv_bn(nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2))
        c2 = self._conv_bn(nn.Conv2d(64, 192, kernel_size=5, padding=2))
        c3 = self._conv(nn.Conv2d(192, 384, kernel_size=3, padding=1))
        c4 = self._conv(nn.Conv2d(384, 256, kernel_size=3, padding=1))
        c5 = self._conv_bn(nn.Conv2d(256, 256, kernel_size=3, padding=1))

        self.conv = nn.Sequential(c1,c2,c3,c4,c5)
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        
        l1 = self._fc(nn.Linear(256*6*6, 4096))
        l2 = self._fc(nn.Linear(4096, 4096))
        
        self.fc = nn.Sequential(l1, l2)
        self.final = nn.Linear(4096, n_classes)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = self.final(x)
        
        print(x.shape)
        
        return x
    
    def _conv_bn(self, conv):
        return nn.Sequential(
            conv,
            self.relu,
            self.maxpool
        )
    
    def _conv(self, conv):
        return nn.Sequential(
            conv,
            self.relu
        )
    
    def _fc(self, linear):
        return nn.Sequential(
            self.dropout,
            linear,
            self.relu
        )
    
def alexnet(n_classes):
    model = AlexNet(n_classes)
    return model

def test():
    x = torch.randn(1,3,224,224)
    model = alexnet(n_classes=3)
    y = model(x)
    
    print("Trainable parameters: ", get_num_parameters(model))
    print("in shape: ", x.shape, ", out shape: ", y.shape)
    
# test()

In [6]:
test()

torch.Size([1, 3])
Trainable parameters:  57016131
in shape:  torch.Size([1, 3, 224, 224]) , out shape:  torch.Size([1, 3])


In [ ]:
def test(convnet="vgg11"):
    x = torch.randn(1,3,224,224)
    
    tests = { 
        "vgg11": vgg11(),
        "vgg11_bn": vgg11_bn(),
        "vgg13": vgg13(),
        "vgg13_bn": vgg13_bn(),
        "vgg16": vgg16(),
        "vgg16_bn": vgg16_bn(),
        "vgg19": vgg19(),
        "vgg19_bn": vgg19_bn(n_classes=3)
    }
    
    model = tests[convnet]
    y = model(x)
    
    print("Trainable parameters: ", get_num_parameters(model))
    print("in shape: ", x.shape, ", out shape: ", y.shape)

#test("vgg19", n_classes = 3)

In [4]:
"""VGG

Adapted from https://github.com/pytorch/vision/blob/master/torchvision/models/vgg.py
Paper: https://arxiv.org/pdf/1409.1556.pdf

Copyright 2021 | farabio
"""
    
import torch
import torch.nn as nn
from typing import Union, List, Any, cast
from farabio.utils.helpers import get_num_parameters

__all__ = [
    'VGG', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
    'vgg19_bn', 'vgg19',
]

cfgs = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
}


class VGG(nn.Module):
    def __init__(self, convnet: nn.Module, n_classes: int = 1000) -> None:
        super(VGG, self).__init__()
        self.conv = convnet
        self.avgpool = nn.AdaptiveAvgPool2d((7,7))
        
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout()
        
        l1 = self._fc(nn.Linear(512*7*7, 4096))
        l2 = self._fc(nn.Linear(4096, 4096))
        
        self.fc = nn.Sequential(l1, l2)
        self.final = nn.Linear(4096, n_classes)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = self.final(x)
        
        return x
    
    def _fc(self, linear):
        return nn.Sequential(
            linear,
            self.relu,
            self.dropout
        )
    
def make_layers(cfg: List[Union[str, int]], batch_norm: bool = False) -> nn.Sequential:
    layers: List[nn.Module] = []
    in_channels = 3
    
    for v in cfg:
        if v == "M":
            v = cast(str, v)
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            v = cast(int, v)
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
        
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            
            in_channels = v
    
    return nn.Sequential(*layers)

def _vgg(cfg: str, batch_norm: bool, **kwargs) -> VGG:
    model = VGG(convnet = make_layers(cfg, batch_norm=batch_norm), **kwargs)
    return model

def vgg11(**kwargs) -> VGG:
    return _vgg(cfgs['A'], False, **kwargs)

def vgg11_bn(**kwargs) -> VGG:
    return _vgg(cfgs['A'], True, **kwargs)

def vgg13(**kwargs) -> VGG:
    return _vgg(cfgs['B'], False, **kwargs)

def vgg13_bn(**kwargs) -> VGG:
    return _vgg(cfgs['B'], True, **kwargs)

def vgg16(**kwargs) -> VGG:
    return _vgg(cfgs['D'], False, **kwargs)

def vgg16_bn(**kwargs) -> VGG:
    return _vgg(cfgs['D'], True, **kwargs)

def vgg19(**kwargs) -> VGG:
    return _vgg(cfgs['E'], False, **kwargs)

def vgg19_bn(**kwargs) -> VGG:
    return _vgg(cfgs['E'], True, **kwargs)

def test(convnet="vgg11"):
    x = torch.randn(1,3,224,224)
    
    tests = { 
        "vgg11": vgg11(),
        "vgg11_bn": vgg11_bn(),
        "vgg13": vgg13(),
        "vgg13_bn": vgg13_bn(),
        "vgg16": vgg16(),
        "vgg16_bn": vgg16_bn(),
        "vgg19": vgg19(),
        "vgg19_bn": vgg19_bn(n_classes=3)
    }
    
    model = tests[convnet]
    y = model(x)
    
    print("Trainable parameters: ", get_num_parameters(model))
    print("in shape: ", x.shape, ", out shape: ", y.shape)

#test("vgg19", n_classes = 3)

In [8]:
test("vgg13")

Trainable parameters:  133047848
in shape:  torch.Size([1, 3, 224, 224]) , out shape:  torch.Size([1, 1000])


In [ ]:
133053736

In [194]:
128778627

128778627

In [187]:
model = vgg11(n_classes=3)


In [188]:
print(pytorch_total_params)

128778627


In [184]:
pytorch_total_params = sum(p.numel() for p in model.parameters())

In [185]:
print(pytorch_total_params)

128778627


In [161]:
_in = torch.Tensor(1,3,224,224)
model = vgg19(n_classes=3)
_out = model(_in)
print(_in.shape, _out.shape)

torch.Size([1, 3, 224, 224]) torch.Size([1, 3])


In [10]:
jaja.append([2])

In [11]:
jaja

[[2]]

In [12]:
jaja.append([12,21])

In [13]:
jaja

[[2], [12, 21]]

In [15]:
hasxa = []

In [16]:
hasxa += [12]

In [17]:
hasxa

[12]

In [18]:
hasxa += [21,21,42]

In [19]:
hasxa

[12, 21, 21, 42]